# High-level MXNet Example

In [1]:
import os
import sys
import numpy as np
import mxnet as mx
import logging
from common.params import *
from common.utils import *

In [2]:
# logging
logger = logging.getLogger();
logger.setLevel(logging.DEBUG);
formatter = logging.Formatter('%(message)s');
h2 = logging.StreamHandler(sys.stdout)
h2.setFormatter(formatter);
logger.addHandler(h2)

In [3]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("MXNet: ", mx.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
MXNet:  0.12.0
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [5]:
def create_symbol(n_classes=N_CLASSES):
    data = mx.symbol.Variable('data')
    # size = [(old-size - kernel + 2*padding)/stride]+1
    # if kernel = 3, pad with 1 either side
    conv1 = mx.symbol.Convolution(data=data, num_filter=50, pad=(1,1), kernel=(3,3))
    relu1 = mx.symbol.Activation(data=conv1, act_type="relu")
    conv2 = mx.symbol.Convolution(data=relu1, num_filter=50, pad=(1,1), kernel=(3,3))
    pool1 = mx.symbol.Pooling(data=conv2, pool_type="max", kernel=(2,2), stride=(2,2))
    relu2 = mx.symbol.Activation(data=pool1, act_type="relu")
    drop1 = mx.symbol.Dropout(data=relu2, p=0.25)
    
    conv3 = mx.symbol.Convolution(data=drop1, num_filter=100, pad=(1,1), kernel=(3,3))
    relu3 = mx.symbol.Activation(data=conv3, act_type="relu")
    conv4 = mx.symbol.Convolution(data=relu3, num_filter=100, pad=(1,1), kernel=(3,3))
    pool2 = mx.symbol.Pooling(data=conv4, pool_type="max", kernel=(2,2), stride=(2,2))
    relu4 = mx.symbol.Activation(data=pool2, act_type="relu")
    drop2 = mx.symbol.Dropout(data=relu4, p=0.25)
           
    flat1 = mx.symbol.Flatten(data=drop2)
    fc1 = mx.symbol.FullyConnected(data=flat1, num_hidden=512)
    relu7 = mx.symbol.Activation(data=fc1, act_type="relu")
    drop4 = mx.symbol.Dropout(data=relu7, p=0.5)
    fc2 = mx.symbol.FullyConnected(data=drop4, num_hidden=n_classes) 
    
    input_y = mx.symbol.Variable('softmax_label')  
    m = mx.symbol.SoftmaxOutput(data=fc2, label=input_y, name="softmax")
    return m

In [6]:
def init_model(m, batchsize=BATCHSIZE, lr=LR, momentum=MOMENTUM):
    ctx = [mx.gpu(0)]
    mod = mx.mod.Module(context=ctx, symbol=m)
    mod.bind(data_shapes=[('data', (batchsize, 3, 32, 32))],
             label_shapes=[('softmax_label', (batchsize,))])
    return mod

In [7]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)
# Load data-iterator
train_iter = mx.io.NDArrayIter(x_train, y_train, BATCHSIZE, shuffle=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 1.07 s, sys: 1.09 s, total: 2.16 s
Wall time: 2.17 s


In [8]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 4.49 ms, sys: 0 ns, total: 4.49 ms
Wall time: 3.58 ms


In [9]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 1 s, sys: 714 ms, total: 1.71 s
Wall time: 1.99 s


In [10]:
%%time
# Main training loop: 49s
model.fit(train_data=train_iter, 
          num_epoch=EPOCHS,
          initializer=mx.init.Xavier(rnd_type='uniform'),
          optimizer='sgd',
          optimizer_params=(('learning_rate', LR), ('momentum', MOMENTUM)),
          eval_metric=mx.metric.create('acc'))

Already bound, ignoring bind()
Epoch[0] Train-accuracy=0.337976
Epoch[0] Time cost=4.913
Epoch[1] Train-accuracy=0.498601
Epoch[1] Time cost=4.840
Epoch[2] Train-accuracy=0.580802
Epoch[2] Time cost=4.886
Epoch[3] Train-accuracy=0.642144
Epoch[3] Time cost=4.821
Epoch[4] Train-accuracy=0.686161
Epoch[4] Time cost=4.836
Epoch[5] Train-accuracy=0.718570
Epoch[5] Time cost=4.835
Epoch[6] Train-accuracy=0.744246
Epoch[6] Time cost=4.849
Epoch[7] Train-accuracy=0.767823
Epoch[7] Time cost=4.830
Epoch[8] Train-accuracy=0.784867
Epoch[8] Time cost=4.836
Epoch[9] Train-accuracy=0.802130
Epoch[9] Time cost=4.828
CPU times: user 44.5 s, sys: 17.3 s, total: 1min 1s
Wall time: 48.6 s


In [11]:
%%time
# Main evaluation loop: 311ms
y_guess = model.predict(mx.io.NDArrayIter(x_test, batch_size=BATCHSIZE, shuffle=False))
y_guess = np.argmax(y_guess.asnumpy(), axis=-1)

CPU times: user 370 ms, sys: 142 ms, total: 512 ms
Wall time: 316 ms


In [12]:
print("Accuracy: ", sum(y_guess == y_test)/len(y_guess))

Accuracy:  0.7707
